# *Predicting Boston Housing Prices using Random Forest*
#### Authors: Tom Sharp, Troy Sattgast

<br>
## Part 0: Environment Setup

In [22]:
# Import the os library and the pandas library (aliased as pd)

import os
import numpy as np
import pandas as pd

Please update the **username** variable with your username on your comp

In [23]:
print(os.getcwd())

C:\Users\tsattgast\Documents\PyScripts\random_forest


In [24]:
username = "tsattgast"

# Store the paths to frequently used files

parent_path = r"C:/Users/" + str(username) + "/Desktop/Demystifying_ML/random_forest/"
data_path = parent_path + "data/Boston Housing Prices.csv"
data_dict_path = parent_path + "data/boston_data_dict.csv"
image_path = parent_path + 'images/'

# ORRRRRR!

parent_path = os.getcwd()
data_path = os.path.join(parent_path,  'data', 'Boston Housing Prices.csv')
data_dict_path = os.path.join(parent_path, 'data', 'boston_data_dict.csv')
image_path = os.path.join(parent_path, 'images')



print(parent_path)
print(data_path)
print(data_dict_path)
print(image_path)

C:\Users\tsattgast\Documents\PyScripts\random_forest
C:\Users\tsattgast\Documents\PyScripts\random_forest\data\Boston Housing Prices.csv
C:\Users\tsattgast\Documents\PyScripts\random_forest\data\boston_data_dict.csv
C:\Users\tsattgast\Documents\PyScripts\random_forest\images


## Part 1: Data Import, Exploration, and Cleaning

During any analysis it is always important to first examine your data. This involves looking at the data itself, the column names, and some summary statistics about the data.

In [25]:
# Read in the data using the pandas package. The data is stored in what is called a dataframe (similar to a spreadsheet)

data = pd.read_csv(data_path)

In [26]:
# Examine number of rows and columns 

print("(num of rows, num of columns) = ", data.shape)

(num of rows, num of columns) =  (506, 17)


In [27]:
# That's a lot of rows. Let's just look at the first three columns of the data, instead of all of them

print(data.head(3))

         town  tract  longitude   latitude    crime  residential  industrial  \
0      Nahant   2011 -70.955002  42.255001  0.00632         18.0        2.31   
1  Swampscott   2021 -70.949997  42.287498  0.02731          0.0        7.07   
2  Swampscott   2022 -70.935997  42.283001  0.02729          0.0        7.07   

  river    nox  rooms      older  distance  highway  tax    ptratio  lstat  \
0    no  0.538  6.575  65.199997    4.0900        1  296  15.300000   4.98   
1    no  0.469  6.421  78.900002    4.9671        2  242  17.799999   9.14   
2    no  0.469  7.185  61.099998    4.9671        2  242  17.799999   4.03   

       cmedv  
0  24.000000  
1  21.600000  
2  34.700001  


In [28]:
# We can list all the column names by calling the "columns" attribute of "data" 
# Def: Attribute - describes the data (an adjective)

print(data.columns)

Index(['town', 'tract', 'longitude', 'latitude', 'crime', 'residential',
       'industrial', 'river', 'nox', 'rooms', 'older', 'distance', 'highway',
       'tax', 'ptratio', 'lstat', 'cmedv'],
      dtype='object')


In [29]:
# We can view summary statistics about the data by calling the "describe()" method of "data"
# Def: Method - take an action on the data (a verb)

print(data.describe())

             tract   longitude    latitude       crime  residential  \
count   506.000000  506.000000  506.000000  506.000000   506.000000   
mean   2700.355731  -71.056389   42.216440    3.613524    11.363636   
std    1380.036830    0.075405    0.061777    8.601545    23.322453   
min       1.000000  -71.289497   42.029999    0.006320     0.000000   
25%    1303.250000  -71.093226   42.180774    0.082045     0.000000   
50%    3393.500000  -71.052902   42.218100    0.256510     0.000000   
75%    3739.750000  -71.019625   42.252249    3.677083    12.500000   
max    5082.000000  -70.809998   42.381000   88.976196   100.000000   

       industrial         nox       rooms       older    distance     highway  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean    11.136779    0.554695    6.284634   68.574901    3.795043    9.549407   
std      6.860353    0.115878    0.702617   28.148862    2.105710    8.707259   
min      0.460000    0.385000    3.5

In [30]:
#What do all these fields mean? Let's use the data dictionary to find out

data_dict = pd.read_csv(data_dict_path)
print(data_dict)

       Col Name                                         Definition
0          town                                       name of town
1         tract                                       census tract
2     longitude                               long of census tract
3      latitude                                lat of census tract
4         crime                      per capita crime rate by town
5   residential  proportion of residential land zoned for lots ...
6    industrial   proportion of non-retail business acres per town
7         river  Charles River dummy variable (= 1 if tract bou...
8           nox  nitric oxides concentration (parts per 10 mill...
9         rooms               average number of rooms per dwelling
10        older  proportion of owner-occupied units older than ...
11     distance  weighted distances to five Boston employment c...
12      highway          index of accessibility to radial highways
13          tax           full-value property-tax rate per $10

**This last value, *cmedv*, is what we would like to predict using a machine learning. Before we can predict, we need to make sure we clean the data.**

In [31]:
# Clean the data - do all of this at once 

data.fillna(0)
data['river'].replace('no', False, inplace = True)
data['river'].replace('yes', True, inplace = True)
data.drop(['town'], axis = 1, inplace = True)
data['val_mean'] = data['cmedv'].mean()

#print(data)
print(data.head(5))

   tract  longitude   latitude    crime  residential  industrial  river  \
0   2011 -70.955002  42.255001  0.00632         18.0        2.31  False   
1   2021 -70.949997  42.287498  0.02731          0.0        7.07  False   
2   2022 -70.935997  42.283001  0.02729          0.0        7.07  False   
3   2031 -70.928001  42.292999  0.03237          0.0        2.18  False   
4   2032 -70.921997  42.298000  0.06905          0.0        2.18  False   

     nox  rooms      older  distance  highway  tax    ptratio  lstat  \
0  0.538  6.575  65.199997    4.0900        1  296  15.300000   4.98   
1  0.469  6.421  78.900002    4.9671        2  242  17.799999   9.14   
2  0.469  7.185  61.099998    4.9671        2  242  17.799999   4.03   
3  0.458  6.998  45.799999    6.0622        3  222  18.700001   2.94   
4  0.458  7.147  54.200001    6.0622        3  222  18.700001   5.33   

       cmedv   val_mean  
0  24.000000  22.528854  
1  21.600000  22.528854  
2  34.700001  22.528854  
3  33.400002

In [32]:
data.head()

,tract,longitude,latitude,crime,residential,industrial,river,nox,rooms,older,distance,highway,tax,ptratio,lstat,cmedv,val_mean
0,2011,-70.955002,42.255001,0.00632,18.0,2.31,False,0.538,6.575,65.199997,4.0900,1,296,15.300000,4.98,24.000000,22.528854
1,2021,-70.949997,42.287498,0.02731,0.0,7.07,False,0.469,6.421,78.900002,4.9671,2,242,17.799999,9.14,21.600000,22.528854
2,2022,-70.935997,42.283001,0.02729,0.0,7.07,False,0.469,7.185,61.099998,4.9671,2,242,17.799999,4.03,34.700001,22.528854
3,2031,-70.928001,42.292999,0.03237,0.0,2.18,False,0.458,6.998,45.799999,6.0622,3,222,18.700001,2.94,33.400002,22.528854
4,2032,-70.921997,42.298000,0.06905,0.0,2.18,False,0.458,7.147,54.200001,6.0622,3,222,18.700001,5.33,36.200001,22.528854


Side Note - In most applications of data science and ML, we wouldtake a closer look at cleaning the data.
Data gathering and cleansing usually consumes +80% of the DS/ML process; however, this dataset happened to be extremely clean when it was retrieved from its source online...and we're pretty lazy.

## Part 2: Decision Tree - The Building Block of Random Forest

In machine learning, the columns in X are referred to as the features, and the y value is referred to as the target.
Since we have a target variable here, this is known as supervised learning.
The split takes the data and splits it into a dataset for training (fitting) the model, and a subset to test the model
Here we are doing an 80/20 split, respectively.


In [33]:
#split the data
features = data.drop('cmedv', axis=1)
print(features.columns.values)
labels = data['cmedv']
feature_list = list(features.columns)


['tract' 'longitude' 'latitude' 'crime' 'residential' 'industrial' 'river'
 'nox' 'rooms' 'older' 'distance' 'highway' 'tax' 'ptratio' 'lstat'
 'val_mean']


In [34]:
print(features)

     tract  longitude   latitude     crime  residential  industrial  river  \
0     2011 -70.955002  42.255001   0.00632         18.0        2.31  False   
1     2021 -70.949997  42.287498   0.02731          0.0        7.07  False   
2     2022 -70.935997  42.283001   0.02729          0.0        7.07  False   
3     2031 -70.928001  42.292999   0.03237          0.0        2.18  False   
4     2032 -70.921997  42.298000   0.06905          0.0        2.18  False   
5     2033 -70.916496  42.304001   0.02985          0.0        2.18  False   
6     2041 -70.935997  42.297001   0.08829         12.5        7.87  False   
7     2042 -70.937500  42.310001   0.14455         12.5        7.87  False   
8     2043 -70.932999  42.312000   0.21124         12.5        7.87  False   
9     2044 -70.929001  42.316002   0.17004         12.5        7.87  False   
10    2045 -70.934998  42.316002   0.22489         12.5        7.87  False   
11    2046 -70.944000  42.317001   0.11747         12.5        7

In [35]:
#convert to numpy arrays
features = np.array(features)
labels = np.array(labels)

In [36]:
# Split the Data into trainnig (for learning) and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state = 1)

#### Before we run our model, let's check our accuracy from our best guess--using an average

In [37]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
test_mean = y_test.mean()
print("The test mean, our baseline for guessing, is {:.2f}".format(test_mean))

baseline_preds = X_test[:, feature_list.index('val_mean')]

# Baseline errors, and display average baseline error and rmse

print(np.sqrt(mean_squared_error(y_test, baseline_preds)))
print(mean_absolute_error(y_test, baseline_preds))

#remove val_mean from features


The test mean, our baseline for guessing, is 22.57
9.94130140151155
6.908447652548243


#### Sci-kit learn uses a 3 step process to train a ML model:
1. Import the model of choice
2. Instantiate the model
3. Fit (train) the model to the training data 

In [38]:
#import
from sklearn import tree

#instantiate 
decision_tree = tree.DecisionTreeRegressor(random_state = 8)

#train/fit
decision_tree = decision_tree.fit(X_train, y_train)

### View the tree as an image

In [39]:
import graphviz 
import pydot

# Change path to images folder
os.chdir(image_path)

dot_data = tree.export_graphviz(decision_tree, out_file = 'tree.dot', feature_names = list(data.drop('cmedv', axis=1).columns), rounded = True, precision = 1)
graph = graphviz.Source(dot_data) 

# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

# Change directory back to parent
os.chdir(parent_path)

ModuleNotFoundError: No module named 'graphviz'

To convert this dot file, you need to use some command line magic I converted the file beforehand, so you can view the tree by running this code block.
For anyone interested, the command line function is below (make sure you are cd'd into the random_forest/images directory and are in teh dm_ml environment)

\> *dot -Tpng tree.dot -o tree.png*

In [40]:
!"images/tree.png"

In [ ]:
# Use the forest's predict method on the test data
predictions = decision_tree.predict(X_test)
pd.DataFrame(predictions, y_test).head(5)

In [ ]:
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics

# accuracy_score(np.array(y_test), np.array(predictions))
rsquare = decision_tree.score(np.array(X_test), np.array(y_test))

print("Our tree's R2 is {:.2} and RMSE is: ".format(rsquare, ))
#print(np.sqrt(metrics.mean_squared_error(X_test, y_test)))

print(X_test.shape)
print(y_test.shape)

Side note - Please note that if we set the random states to different numbers, the result would be different, however
While this decision tree is quite accurate, we can possibly improve accuracy using the random forest model
The random forest model essentially builds multiple decision trees, takes the outputs from all of those trees, and 
determines the best prediction by taking the average (regression) or the mode (classification) of the outputs 

#### This seems like a pretty good accuracy, except we overfit the model...

In [ ]:
from IPython.display import Image
Image(filename= str(image_path) + 'overfitting_underfitting.png')

There are ways to combat overfitting by tuning the model. One way to do this is decrease the depth of the tree (either during or after fitting - research *pruning*). We won't get into that here, instead we will show another way to more accurately (and powerfully) predict our outcomes - the random forest.

## Part 3: Random Forest

In [ ]:
Image(filename= str(image_path) + 'random-forest.jpg')

In [ ]:
# Again, we import, instantiate, and then fit
# Here, n_estimators is the number of decision trees in our random forest

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 10)
rf.fit(X_train, y_train)

In [ ]:
# Now let's predict and see the predictions next to the actual y values

predictions = rf.predict(X_test)
pd.DataFrame(y_test, predictions).head()

In [ ]:
from sklearn.metrics import accuracy_score

# accuracy_score(np.array(y_test), np.array(predictions))
accuracy = rf.score(np.array(X_test), np.array(y_test))

print("Our tree's accuracy is " + str(round(accuracy*100,2)) + "%")

Our accuracy improved! Let's take a deeper look at how close our model is to predicting actual housing prices

In [ ]:
# Calculate the average error for the predicted results
absolute_errors = abs(predictions - y_test)
mean_absolute_error = np.mean(absolute_errors)
mean_absolute_error = round(mean_absolute_error, 4)

# Remember that our housing prices are in thousands of dollars, so let's show that here
print('Mean Absolute Error: $', 1000*mean_absolute_error, sep = '')

In [ ]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot

# Pull out one tree from the forest
tree = rf.estimators_[5]

os.chdir(image_path)

# Export the image to a dot file
feature_list = data.drop('cmedv', axis=1).columns
export_graphviz(tree, out_file = 'rf_tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('rf_tree.dot')

os.chdir(parent_path)

In [ ]:
!("images/tree_from_random_forest_output.png")

Let's see which factors of a neighborhood influence it's price the most. We can do this using a few more complex techniques in Python. I won't be getting into these and I am also going to use some code that was written by William Koehrsen in his article that can be found here: https://towardsdatascience.com/random-forest-in-python-24d0893d51c0

In [ ]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

It looks like lstat and rooms make up about 80% of the importance in predicting the housing price.<br>
Does this make sense? It is always improtant to look at your model output and determine if it logically matches the context of the problem

## Part 4 - Model Simplification

In [ ]:
sorted_importances = [importance[1] for importance in feature_importances]
sorted_features = [importance[0] for importance in feature_importances]
cumulative_importances = np.round(np.cumsum(sorted_importances),2)

importance_df = pd.DataFrame({'features': sorted_features, 'importance': sorted_importances, 'cumulative importance': cumulative_importances})
importance_df

In [ ]:
# We can pick what we want our cumulative importance to be. Here I chose 95%
new_df = importance_df[importance_df['cumulative importance'] <= 0.95]
new_df

In [ ]:
# Let's say we only want to use these features. We can re-run the random forest with only these

#split the data
features = data[new_df.features]
labels = data['cmedv']
print(features.columns)

#convert to numpy arrays
import numpy as np
features = np.array(features)
labels = np.array(labels)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state = 1)

In [ ]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 10)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)

# accuracy_score(np.array(y_test), np.array(predictions))
accuracy = rf.score(np.array(X_test), np.array(y_test))
print("Our tree's accuracy is " + str(round(accuracy*100,2)) + "%")

# Calculate the average error for the predicted results
absolute_errors = abs(predictions - y_test)
mean_absolute_error = np.mean(absolute_errors)
mean_absolute_error = round(mean_absolute_error, 4)
print('Mean Absolute Error: $', 1000*mean_absolute_error, sep = '')

Remember that our accuracy before was ~ 92% and that the MAE was about $2100 as well
While our new model may seem just as good as before, remember that we dropped more than half of our features. 
That should prove that the other features really weren't adding much more value.
This is just one example of how models do not have super complex to provide good results. What we also gained here is decreased runtime - it took less computation time to get almost the same accuracy. When scaling a model to production, this is a trade-off we would likely want to make.

## Part 5 - Tuning the Forest

"While model parameters are learned during training — such as the slope and intercept in a linear regression — hyperparameters must be set by the data scientist before training." - William Koehrsen
<br>


<br>
<br>
## Resources 

Data Source <br>
https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

Jupyter <br>
https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html

Pandas <br>
https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python

Information Gain and Entropy <br>
https://www.saedsayad.com/decision_tree.htm <br>

How to Become a Data Science <br>
http://www.nerdgraph.com/new-infographic-become-data-scientist-8-steps/ <br>
https://towardsdatascience.com/how-to-learn-data-science-if-youre-broke-7ecc408b53c7 <br>
https://www.class-central.com/subject/data-science <br>

General Sources & Good Reads <br>
https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6 <br>
https://medium.com/machine-learning-for-humans/why-machine-learning-matters-6164faf1df12 <br>
https://medium.com/@williamkoehrsen/random-forest-simple-explanation-377895a60d2d <br>
https://towardsdatascience.com/random-forest-in-python-24d0893d51c0 <br>
https://github.com/WillKoehrsen/Data-Analysis/tree/master/random_forest_explained <br>
https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74